In [1]:
import pandas as pd
import sqlite3
import pickle
from collections import defaultdict
#run this code on a server, laptop will not be able to handle it. loop of 11,000*8,00,000

In [ ]:
conn = sqlite3.connect("Extracted.db")
c = conn.cursor()

In [ ]:
sample = pd.read_sql("SELECT * FROM Sample", conn)

In [ ]:
graph = pd.read_sql("SELECT * FROM Graph", conn)

In [ ]:
metapath2 = pd.read_sql("SELECT * FROM metapath2", conn)

In [ ]:
metapath3 = pd.read_sql("SELECT * FROM metapath3", conn)

In [ ]:
metapath4 = pd.read_sql("SELECT * FROM metapath4", conn)

In [ ]:
with open('CNGG_count.pickle', 'rb') as handle:
    CNGG_count = pickle.load(handle)

In [ ]:
with open('CNG_count.pickle', 'rb') as handle:
    CNG_count = pickle.load(handle)

In [ ]:
with open('Word_count.pickle', 'rb') as handle:
    Word_count = pickle.load(handle)

In [ ]:
with open('Lemma_count.pickle', 'rb') as handle:
    Lemma_count = pickle.load(handle)

In [ ]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [ ]:
innerdict = merge_two_dicts(CNG_count, CNGG_count)
#we merge them for easier selection in the future
#as denominators in metapath3,4 are either count of CNG or CNG_Group

In [ ]:
features = defaultdict(list)
global features

In [ ]:
# use threading for the following operations, on a server

def mp2(metapath2, sample):

    for mp2i,mp2r in metapath2.iterrows():
        #iterating through all metapaths of size 2
        node1 = mp2r['node1']
        node2 = mp2r['node2']
        #since it is a metapath of size 2 we have only 2 nodes

        vector = []

        for i,r in sample.iterrows():
            #iterating through 11,000 samples we selected
            den = 0
            #initializing denominator of bigram probability to 0
            
            #checking to see what type of External nodes are there in the metapath
            #assigning den as per type of node
            
            if node1 == 'Lemma':
                value1 = r['Entity_1'].split("_")[0]
                den = Lemma_count[value1]
            elif node1 == 'CNG':
                value1 = r['Entity_1'].split("_")[1]
                den = CNG_count[int(value1)]
            else:
                value1 = r['Entity_1']
                den = Word_count[value1]


            if node2 == 'Lemma':
                value2 = r['Entity_2'].split("_")[0]
            elif node2 == 'CNG':
                value2 = r['Entity_2'].split("_")[1]
            else:
                value2 = r['Entity_2']        

            if den == 0:
                #this is okay as we will be considering metapaths with maximum unique values 
                prob12 = 0
            else:
                
                temp = graph.loc[ (graph['Entity_1'] == value1) & (graph['Entity_2'] == value2)] 
                #temp is used to get numeratr of bigram probabily from Graph
#               temp = pd.read_sql("SELECT Weight FROM Graph WHERE Entity_1 = '%s' AND Entity_2 = '%s' " %(value1, value2))
                prob12 = temp.Weight[0]/den

                prob = prob12

            vector.append(prob)

        features[node1+'|'+node2] = vector
#Important: Word|Word column vector is our label vector ! for Mutual Information Regression

In [ ]:

def mp3(metapath3, sample):
    
    for mp3i,mp3r in metapath3.iterrows():
        #iterating through all metapaths of size 3
        node1 = mp3r['node1']
        node2 = mp3r['node2']
        node3 = mp3r['node3']
        #since it is a metapath of size 3 we have only 3 nodes
        vector = []

        for i,r in sample.iterrows():
            den1 = 0
            den2 = 0

            if node1 == 'Lemma':
                value1 = r['Entity_1'].split("_")[0]
                den1 = Lemma_count[value1]
            elif node1 == 'CNG':
                value1 = r['Entity_1'].split("_")[1]
                den1 = CNG_count[int(value1)]
            else:
                value1 = r['Entity_1']
                den1 = Word_count[value1]

            value2 = node2

            if node3 == 'Lemma':
                value3 = r['Entity_2'].split("_")[0]
            elif node3 == 'CNG':
                value3 = r['Entity_2'].split("_")[1]
            else:
                value3 = r['Entity_2']

            den2 = innerdict[node2]
            #selecting from innerdict as it has counts of both CNG and CNG_Groups

            if den1 == 0:
                prob12 = 0
            else:
                temp = graph.loc[ (graph['Entity_1'] == value1) & (graph['Entity_2'] == value2)] 
#               temp = pd.read_sql("SELECT Weight FROM Graph WHERE Entity_1 = %s AND Entity_2 = %s" %(value1, value2))
                prob12 = temp.Weight[0]/den1

            if den2 == 0:
                prob23 = 0
                
            else:
                temp = graph.loc[ (graph['Entity_1'] == value2) & (graph['Entity_2'] == value3)] 
#               temp = pd.read_sql("SELECT Weight FROM Graph WHERE Entity_1 = %s AND Entity_2 = %s" %(value2, value3))
                prob23 = temp.Weight[0]/den2

            prob = prob12*prob23

            vector.append(prob)

        features[node1 + '|' + node2 + '|'+ node3]  = vector


In [ ]:
def mp4(metapath4, sample):
    

    for mp4i,mp4r in metapath4.iterrows():
        #iterating through all metapaths of size 4
        node1 = mp4r['node1']
        node2 = mp4r['node2']
        node3 = mp4r['node3']
        node4 = mp4r['node4']
        
        #since it is a metapath of size 4 we have only 4 nodes
        vector = []

        for i,r in sample.iterrows():
            den1 = 0
            den2 = 0
            den3 = 0

            if node1 == 'Lemma':
                value1 = r['Entity_1'].split("_")[0]
                den1 = Lemma_count[value1]
            elif node1 == 'CNG':
                value1 = r['Entity_1'].split("_")[1]
                den1 = CNG_count[int(value1)]
            else:
                value1 = r['Entity_1']
                den1 = Word_count[value1]

            value2 = node2

            value3 = node3

            if node4 == 'Lemma':
                value4 = r['Entity_2'].split("_")[0]
            elif node4 == 'CNG':
                value4 = r['Entity_2'].split("_")[1]
            else:
                value4 = r['Entity_2']

            den2 = innerdict[node2]
            den3 = innerdict[node3]

            if den1 == 0:
                prob12 = 0
            else:
                temp = graph.loc[ (graph['Entity_1'] == value1) & (graph['Entity_2'] == value2)] 
#               temp = pd.read_sql("SELECT Weight FROM Graph WHERE Entity_1 = %s AND Entity_2 = %s" %(value1, value2))
                prob12 = temp.Weight[0]/den1

            if den2 == 0:
                prob23 = 0
            else:
                temp = graph.loc[ (graph['Entity_1'] == value2) & (graph['Entity_2'] == value3)] 
#               temp = pd.read_sql("SELECT Weight FROM Graph WHERE Entity_1 = %s AND Entity_2 = %s" %(value2, value3))
                prob23 = temp.Weight[0]/den2

            if den3 == 0:
                prob34 = 0
            else:
                temp = graph.loc[ (graph['Entity_1'] == value3) & (graph['Entity_2'] == value4)] 
#               temp = pd.read_sql("SELECT Weight FROM Graph WHERE Entity_1 = %s AND Entity_2 = %s" %(value3, value4))
                prob34 = temp.Weight[0]/den3


            prob = prob12*prob23*prob34

            vector.append(prob)

        features[node1 + '|' + node2 + '|'+ node3 + '|' +node4]  = vector


In [ ]:
#Threading begins here
#Calculate Length of Metapahts so that we can slpit them for threading 
lenmp2 = len(metapath2.node1)
lenmp3 = len(metapath3.node1)
lenmp4 = len(metapath4.node1)

In [ ]:
#Running a total of 20 threads , mp2 : 1 , mp3: 5, mp4 : 14 threads respectivly 

step3 = int(lenmp3/5)
step4 = int(lenmp4/14)
ind3 = []
z =0
for i in range(4):
    ind3.append([z, z+step3])
    z = z+step3
ind3.append([z, lenmp3-1])

ind4=[]
z =0
for i in range(13):
    ind4.append([z, z+step4])
    z = z+step4
ind4.append([z, lenmp4-1])

    
t1 = threading.Thread(target=mp2, args=(metapath2, sample))

t2 = threading.Thread(target=mp3, args=(metapath3[ind3[0][0]:ind3[0][1], :], sample))
t3 = threading.Thread(target=mp3, args=(metapath3[ind3[1][0]:ind3[1][1], :], sample))
t4 = threading.Thread(target=mp3, args=(metapath3[ind3[2][0]:ind3[2][1], :], sample))
t5 = threading.Thread(target=mp3, args=(metapath3[ind3[3][0]:ind3[3][1], :], sample))
t6 = threading.Thread(target=mp3, args=(metapath3[ind3[4][0]:ind3[4][1], :], sample))

t7 = threading.Thread(target=mp4, args=(metapath4[ind4[0][0]:ind4[0][1], :], sample))
t8 = threading.Thread(target=mp4, args=(metapath4[ind4[1][0]:ind4[1][1], :], sample))
t9 = threading.Thread(target=mp4, args=(metapath4[ind4[2][0]:ind4[2][1], :], sample))
t10 = threading.Thread(target=mp4, args=(metapath4[ind4[3][0]:ind4[3][1], :], sample))
t11 = threading.Thread(target=mp4, args=(metapath4[ind4[4][0]:ind4[4][1], :], sample))
t12 = threading.Thread(target=mp4, args=(metapath4[ind4[5][0]:ind4[5][1], :], sample))
t13 = threading.Thread(target=mp4, args=(metapath4[ind4[6][0]:ind4[6][1], :], sample))
t14 = threading.Thread(target=mp4, args=(metapath4[ind4[7][0]:ind4[7][1], :], sample))
t15 = threading.Thread(target=mp4, args=(metapath4[ind4[8][0]:ind4[8][1], :], sample))
t16 = threading.Thread(target=mp4, args=(metapath4[ind4[9][0]:ind4[9][1], :], sample))
t17 = threading.Thread(target=mp4, args=(metapath4[ind4[10][0]:ind4[10][1], :], sample))
t18 = threading.Thread(target=mp4, args=(metapath4[ind4[11][0]:ind4[11][1], :], sample))
t19 = threading.Thread(target=mp4, args=(metapath4[ind4[12][0]:ind4[12][1], :], sample))
t20 = threading.Thread(target=mp4, args=(metapath4[ind4[13][0]:ind4[13][1], :], sample))


t1.start()
t2.start()
t3.start()
t4.start()
t5.start()
t6.start()
t7.start()
t8.start()
t9.start()
t10.start()
t11.start()
t12.start()
t13.start()
t14.start()
t15.start()
t16.start()
t17.start()
t18.start()
t19.start()
t20.start()

t1.join()
t2.join()
t3.join()
t4.join()
t5.join()
t6.join()
t7.join()
t8.join()
t9.join()
t10.join()
t11.join()
t12.join()
t13.join()
t14.join()
t15.join()
t16.join()
t17.join()
t18.join()
t19.join()
t20.join()

#tip : use ctrl + left mouse click to have cursir select multiple lines and change at once

In [ ]:
conn1 = sqlite3.connect("Features.db")

In [ ]:
dataframe = pd.DataFrame(features, index = [i for i in range(110001)]) 

In [ ]:
dataframe.to_sql("FeaturesRaw", conn1)

In [ ]:
conn.close()
conn1.close()
